<a href="https://colab.research.google.com/github/RemiCailliot/Deep_Learning_Project/blob/main/lstm_by_epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

In [ ]:
filename = "/content/drive/MyDrive/Colab files/001ssb.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
text = raw_text.lower()

In [ ]:
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



Corpus length: 1607894
Total chars: 52
Number of sequences: 535952


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [ ]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



In [ ]:
epochs = 20
batch_size = 128


model.fit(x, y, batch_size=batch_size, epochs=epochs)

Epoch 1/20
4188/4188 [==============================] - 59s 13ms/step - loss: 1.6962
Epoch 2/20
4188/4188 [==============================] - 56s 13ms/step - loss: 1.4794
Epoch 3/20
4188/4188 [==============================] - 55s 13ms/step - loss: 1.4303
Epoch 4/20
4188/4188 [==============================] - 55s 13ms/step - loss: 1.4015
Epoch 5/20
4188/4188 [==============================] - 56s 13ms/step - loss: 1.3840
Epoch 6/20
4188/4188 [==============================] - 57s 14ms/step - loss: 1.3708
Epoch 7/20
4188/4188 [==============================] - 57s 14ms/step - loss: 1.3625
Epoch 8/20
4188/4188 [==============================] - 57s 13ms/step - loss: 1.3545
Epoch 9/20
4188/4188 [==============================] - 57s 14ms/step - loss: 1.3469
Epoch 10/20
4188/4188 [==============================] - 56s 13ms/step - loss: 1.3423
Epoch 11/20
4188/4188 [==============================] - 56s 13ms/step - loss: 1.3374
Epoch 12/20
4188/4188 [==============================] - 56s 13

In [ ]:
print()
print("Generating text after epoch: %d" % epochs)
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5]:
    print("...Diversity:", diversity)

    generated = ""
    sentence = text[start_index : start_index + maxlen]
    print('...Generating with seed: "' + sentence + '"')

    for i in range(200):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

        #print("...Generated: ", generated)
    print()
    print("...Generated: ", generated)


Generating text after epoch: 20
...Diversity: 0.2
...Generating with seed: " wedding ride.  his hair had never been "

...Generated:  a stark to the stark stark of the stark of the stark of the stark of the stark of the strong and stark and stark. the common said as the stark stark was the stark of the stark of the stark of the ston
...Diversity: 0.5
...Generating with seed: " wedding ride.  his hair had never been "

...Generated:  hadow a horse and said  the wast and seemed with the second.  "you can be a stup too men seemed to take the dirtsters and the armory, the first stark and light and made her wordion of the senting room


In [ ]:
model.evaluate()